# Unfooded Country Analysis from FAO R&D's department  

## 1. Data preprocessing

    1.1 Loading natives data with Pandas
        1.1.1 Food resources availibility's by countries by product
        1.1.2 Amout of food resources given to countries in food deficit
        1.1.3 Food insecurity describes by the number of undernurished people
        1.1.4 Population by countries
    1.2 Have a quick look to our data distributions
    1.3 How to deal with missing values

## 2. Compute data for answering to Marc's requests

    2.1 Unnourished people proportion in 2017
    2.2 Number of people which can be fooded with global food stock
    2.3 Number of people which can be fooded with vegetables
    2.4 The use of global food stock across each countries


## 3. Compute data for answering to Mélanie's requests

    3.1 Solution for request 1
    3.2 Solution for request 2
    3.3 Solution for request 3

# 1. Data preprocessing
## 1.1 Loading natives data into Python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1.1.1 Availibility of food resources by countries by type of product

In [ ]:
dispo_alim = pd.read_csv('P4OC - FAO/dispo_alimentaire.csv', header=0, sep=",")

In [ ]:
dispo_alim

In [ ]:
dispo_alim.isna().describe()

In [ ]:
dispo_alim_animaux = dispo_alim[dispo_alim['Aliments pour animaux'].isna() == False]

In [ ]:
dispo_alim_copy = dispo_alim.copy()

In [ ]:
dispo_alim_copy = dispo_alim_copy.replace(np.nan, 0)

In [ ]:
dispo_alim_copy[dispo_alim_copy['Produit'] == 'Abats Comestible']

In [ ]:
dispo_alim_copy[dispo_alim_copy['Produit'] == 'Abats Comestible']['Disponibilité alimentaire (Kcal/personne/jour)'].plot.hist()
plt.show()

### 1.1.2 Food resources given to countries in food deficit

In [ ]:
aide_alim = pd.read_csv('P4OC - FAO/aide_alimentaire.csv', header=0, sep=',')

In [ ]:
aide_alim.head(10)

In [ ]:
aide_alim.sort_values(['Pays bénéficiaire','Année'], ascending=True)

### 1.2.3 Food insecurity describe by the number of undernurished people

In [ ]:
col_types = {'Zone' : str, 'Année' : str, 'Valeur' : int}
sous_alim = pd.read_csv('P4OC - FAO/sous_nutrition.csv', header=0, sep=',')

In [ ]:
sous_alim['Valeur'].isna().describe()

In [ ]:
sous_alim.head(5)

In [ ]:
sous_alim['Valeur'].isna()

In [ ]:
# Suppression des lignes avec des valeurs manquantes
sous_alim[sous_alim['Valeur'].isna()==False]

### 1.2.4 Population by countries

In [ ]:
pop_pays = pd.read_csv('P4OC - FAO/population.csv', header=0, sep=',')

In [ ]:
pop_pays.head(5)

In [ ]:
pop_pays[pop_pays['Année'] == 2017].set_index('Zone').head(5).plot.bar(y='Valeur')

In [ ]:
pop_pays.isna().describe()

## 1.3 How to deal with missing values

### 1.3.1 Food Unsecurity's data set

In [ ]:
# Le nombre totale de personne en sous nutrition 
col_types = {'Zone' : str, 'Année' : str, 'Valeur' : np.float64}
sous_alim.dtypes

In [ ]:
sous_alim[sous_alim['Valeur'] == '<0.1']['Valeur']

In [ ]:
# convert string into float
def convert_string_to_float(value):
    
    if value == '<0.1':
        return 0.1
    else :
        return np.float64(value)
    

In [ ]:
sous_alim_copy = sous_alim.copy()

In [ ]:
# proportion d'occurence '<0.1' dans Valeur 
sous_alim['Valeur'][sous_alim['Valeur'] == '<0.1'].count() / sous_alim['Valeur'].count()

In [ ]:
# replace nan value and convert string
sous_alim_copy['Valeur'] = sous_alim_copy['Valeur'].apply(convert_string_to_float).replace(np.nan, 0)

## 2.1 Unnourished people proportion in 2017

In [ ]:
# La population mondiale en 2017
tot_pop = pop_pays['Valeur'][pop_pays['Année'] == 2017].sum()

In [ ]:
# Population total en sous nutrition
tot_pop_sous_alim = sous_alim_copy['Valeur'][sous_alim_copy['Année'] == '2016-2018'].sum()*1000

In [ ]:
tot_pop_sous_alim

In [ ]:
prop_sous_alim = tot_pop_sous_alim / tot_pop

In [ ]:
prop_sous_alim

In [ ]:
size_prop_sous = [prop_sous_alim, 1-prop_sous_alim]

In [ ]:
ranking_sous_alim_pays = sous_alim_copy[sous_alim_copy['Année'] == '2016-2018'][['Zone','Valeur']].sort_values(by='Valeur',ascending=False).head(10)

In [ ]:
plt.pie(size_prop_sous, labels=['Pop sous alimentée', 'Pop en suffisance alimentaire'], explode = (0.2,0), autopct='%1.1f%%', shadow = True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
sous_alim_pop_total = sous_alim_copy[sous_alim_copy['Année'] == '2016-2018']['Valeur'].sum()
size = list(ranking_sous_alim_pays.Valeur/sous_alim_pop_total)
autre_country = 1 - sum(size)

In [ ]:
labels = list(ranking_sous_alim_pays.Zone)
size.insert(0,autre_country)
labels.insert(0,'autres')

In [ ]:
fig1, ax1  = plt.subplots(figsize=(5,6), subplot_kw=dict(aspect="equal"))
ax1.pie(size, labels=labels, autopct='%1.1f%%',
       shadow = True, textprops=dict(color="w", weight="bold"))
ax1.set_title("10 most unnourished countries in the world")
ax1.legend(labels,
        loc='center left',        
        bbox_to_anchor=(1, 0, 0.5, 1))
plt.show()


In [ ]:
del size, labels

In [ ]:
def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:.1f}%\n({:d} g)".format(pct, absolute)

## 2.2 Volume of people which can be fooded with global food stock

In [ ]:
dispo_alim_copy = dispo_alim.copy()

In [ ]:
dispo_alim.describe()

In [ ]:
dispo_alim_copy['Disponibilité intérieure'] = dispo_alim_copy['Disponibilité intérieure'].replace(np.nan, 0)

In [ ]:
volume_dispo_global = dispo_alim_copy['Disponibilité alimentaire (Kcal/personne/jour)'].sum()

In [ ]:
dispo_alim_total_pays = dispo_alim_copy[['Zone', 'Disponibilité alimentaire (Kcal/personne/jour)']].groupby('Zone').sum().reset_index()

In [ ]:
pop_pays_2017

In [ ]:
pop_pays_2017 = pop_pays[pop_pays['Année'] == 2017][['Zone','Valeur']]

In [ ]:
capacite_alim_pays = dispo_alim_total_pays.join(pop_pays_2017.set_index('Zone'), 'Zone').rename({'Disponibilité alimentaire (Kcal/personne/jour)' : 'Disponibilité alimentaire totale (Kcal/personne/jour)'})

In [ ]:
capacite_alim_pays['Disponibilité alimentaire totale'] = capacite_alim_pays['Disponibilité alimentaire (Kcal/personne/jour)'] * capacite_alim_pays['Valeur'] * 1000

In [ ]:
# Disponibilité alimentaire mondiale en Kcal
dispo_alim_total = capacite_alim_pays['Disponibilité alimentaire totale'].sum()
# Apport journalier d'une personne adulte estimé à 2500 Kcal
nb_theorique_pers = dispo_alim_total / 2500

In [ ]:
nb_theorique_pers

## 2.3 Number of people which can be fooded with vegetables

In [ ]:
dispo_alim_total_pays_vege = dispo_alim[dispo_alim['Origine'] == 'vegetale'][['Zone','Disponibilité alimentaire (Kcal/personne/jour)']].groupby('Zone').sum().reset_index()

In [ ]:
capacite_alim_pays_vege = dispo_alim_total_pays_vege.join(pop_pays_2017.set_index('Zone'), 'Zone')

In [ ]:
capacite_alim_pays_vege

In [ ]:
capacite_alim_pays_vege['Disponibilité Alimentaire totale'] = capacite_alim_pays_vege['Disponibilité alimentaire (Kcal/personne/jour)'] * capacite_alim_pays_vege['Valeur'] * 1000
dispo_total_alim_vege = capacite_alim_pays_vege['Disponibilité Alimentaire totale'].sum()
nb_theorique_pers_vege = dispo_total_alim_vege / 2500

In [ ]:
nb_theorique_pers_vege

In [ ]:
# écrire une fonction pour scaler le résultat produit végétale et produit animale

def get_theorical_number(dispo_alim : pd.DataFrame, pop_pays : pd.DataFrame, origine='all') -> float :
        
        if origine != 'all':
            dispo_alim_pays = dispo_alim[dispo_alim['Origine'] == origine][['Zone', 'Disponibilité alimentaire (Kcal/personne/jour)']].groupby('Zone').sum()
        else :
            dispo_alim_pays = dispo_alim[['Zone', 'Disponibilité alimentaire (Kcal/personne/jour)']].groupby('Zone').sum()
        capacite_alim_pays = pop_pays.join(dispo_alim_pays, 'Zone')
        capacite_alim_pays['Disponibilité alimentaire totale'] = capacite_alim_pays['Disponibilité alimentaire (Kcal/personne/jour)'] * capacite_alim_pays['Valeur'] * 1000
        dispo_totale_alim = capacite_alim_pays['Disponibilité alimentaire totale'].sum()
        theorical_number = dispo_totale_alim / 2500
        return theorical_number

In [ ]:
get_theorical_number(dispo_alim, pop_pays_2017)

## 2.4 The use of the world food stock 

In [ ]:
disp_int = aliment_animaux + autre_utilisation + alim_humaine + export - perte

In [ ]:
dispo_alim_copy[['Aliments pour animaux', 'Autres Utilisations', 'Exportations - Quantité','Nourriture', 'Pertes']].sum()

In [ ]:

dispo_alim_copy['Disponibilité intérieure'].sum()

In [ ]:
sum(dispo_alim_copy[['Aliments pour animaux', 'Autres Utilisations', 'Exportations - Quantité','Nourriture', 'Pertes']].sum())

In [ ]:
aide_alim['Valeur'].sum()

In [ ]:

8851382.0 - 453698.0